In [116]:
import numpy as np
import copy

In [373]:
class PolicyIteration:
    def __init__(self, height, width, gamma=1, terminals=[(0, 0)]):
        self.actions = ("U", "D", "L", "R")
        self.reward = -1
        self.width = width
        self.height = height
        self.states = []
        for h in range(height):
            for w in range(width):
                self.states += [(h, w)]
        self.state_values = {s:0 for s in self.states}
        self.gamma = gamma
        self.policy = {s:self.actions for s in self.states}
        
        
        self.set_terminals(terminals)
        
        self.cal_action_prob()
    
    
    def set_terminals(self, terminals):
        for t in terminals:
            self.policy[t] = []
    
    def cal_action_prob(self):
        self.action_prob = {}
        for s in self.states:
            self.action_prob[s] = {}
            for a in self.policy[s]:
                self.action_prob[s][a] = 1 / len(self.policy[s])

    def try_action(self, state, action):
        pos_y, pos_x = state
        if action == "U":
            if pos_y-1 >= 0:
                pos_y -= 1
        elif action == "D":
            if pos_y+1 < self.height:
                pos_y += 1
        elif action == "L":
            if pos_x-1 >= 0:
                pos_x -= 1
        elif action == "R":
            if pos_x+1 < self.width:
                pos_x += 1
            
        return self.action_prob[state][action], [(pos_y, pos_x)], [1]
    
    
    def policy_evaluation(self):
        theta = 1
        delta = 10
        
        idx = 0
        pre_state_values = copy.deepcopy(self.state_values)
        while delta > theta:
            delta = 0
            tmp_state_values = copy.deepcopy(pre_state_values)
            for s in self.states:
                action_value = 0
                action_prob = 0
                for a in self.policy[s]:
                    action_prob, next_states, transition_probs = self.try_action(s, a)                        
                    for next_state, trans_prob in zip(next_states, transition_probs):
                        action_value += trans_prob * (self.reward + self.gamma * pre_state_values[next_state])
                
                tmp_state_values[s] = action_prob * action_value
                delta = max(delta, abs(tmp_state_values[s]-pre_state_values[s]))
                
            pre_state_values = copy.deepcopy(tmp_state_values)
            idx += 1
            if idx > 10:
                break
        self.state_values = copy.deepcopy(tmp_state_values)
        
    def policy_improvement(self):
        policy_stable = True
        for s in self.states:
            action_values = {}
            for a in self.actions:
                if a in self.policy[s]:
                    action_prob, next_states, transition_probs = self.try_action(s, a)
                    for next_state, trans_prob in zip(next_states, transition_probs):
                        action_values[a] = trans_prob * (self.reward + self.gamma * self.state_values[next_state])
                else:
                    action_values[a] = -np.inf
               
            old_policy = self.policy[s]
            max_action_value = max(action_values.values())
            if max_action_value != -np.inf:
                self.policy[s] = [a for a in self.actions if action_values[a]==max_action_value]
                if old_policy != self.policy[s]:
                    policy_stable = False
                    
        self.cal_action_prob()
        
        return policy_stable
            
     
    def perform(self, iter_num):
        print("initial")
        policy_iteration.print_policy()
        policy_iteration.print_state_values()
            
        for i in range(iter_num):
            print("\nstep: ", i+1)
            self.policy_evaluation()

            self.print_state_values()

            policy_stable = self.policy_improvement()
            self.print_policy()
            
            #self.print_action_prob()
            
            if policy_stable == True:
                break
    
    def print_policy(self):
        print("policy")
        for h in range(self.height):
            ss = []
            for w in range(self.width):
                action = "".join([x if x in self.policy[(h, w)] else "_" for x in self.actions])

                ss += [action]
            print(ss)
            
    def print_action_prob(self):
        print("action_prob: ", self.action_prob)
        
    def print_state_values(self):
        print("state values")
        for h in range(self.height):
            sv = []
            for w in range(self.width):
                sv += [f"{self.state_values[(h, w)]:+1.1f}"]
            print(sv)

In [374]:
h, w = 4, 4
policy_iteration = PolicyIteration(h, w, terminals=((0, 0), (h-1, w-1)))

In [375]:
policy_iteration.perform(10)

initial
policy
['____', 'UDLR', 'UDLR', 'UDLR']
['UDLR', 'UDLR', 'UDLR', 'UDLR']
['UDLR', 'UDLR', 'UDLR', 'UDLR']
['UDLR', 'UDLR', 'UDLR', '____']
state values
['+0.0', '+0.0', '+0.0', '+0.0']
['+0.0', '+0.0', '+0.0', '+0.0']
['+0.0', '+0.0', '+0.0', '+0.0']
['+0.0', '+0.0', '+0.0', '+0.0']

step:  1
state values
['+0.0', '-1.0', '-1.0', '-1.0']
['-1.0', '-1.0', '-1.0', '-1.0']
['-1.0', '-1.0', '-1.0', '-1.0']
['-1.0', '-1.0', '-1.0', '+0.0']
policy
['____', '__L_', 'UDLR', 'UDLR']
['U___', 'UDLR', 'UDLR', 'UDLR']
['UDLR', 'UDLR', 'UDLR', '_D__']
['UDLR', 'UDLR', '___R', '____']

step:  2
state values
['+0.0', '-1.0', '-2.0', '-2.0']
['-1.0', '-2.0', '-2.0', '-2.0']
['-2.0', '-2.0', '-2.0', '-1.0']
['-2.0', '-2.0', '-1.0', '+0.0']
policy
['____', '__L_', '__L_', 'UDLR']
['U___', 'U_L_', 'UDLR', '_D__']
['U___', 'UDLR', '_D_R', '_D__']
['UDLR', '___R', '___R', '____']

step:  3
state values
['+0.0', '-1.0', '-2.0', '-3.0']
['-1.0', '-2.0', '-3.0', '-2.0']
['-2.0', '-3.0', '-2.0', '-1.0'